In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

ImportError: DLL load failed: The specified module could not be found.

# Auxiliary Functions

In [2]:
def getExcelFrame(**kwargs):
    return pd.read_clipboard(**dict(kwargs, header = kwargs.get("header", None))).as_matrix()

In [3]:
def pasteExcelFrame(x, header = False, index = False):
    pd.DataFrame(x).to_clipboard(header, index)

In [40]:
def getLHNormals(intervals, dimensions):
    U = np.array([(np.random.permutation(intervals) + np.random.uniform(size = intervals))/intervals for _ in range(dimensions) ]).T

# Bank Account

## Naive Simulation

In [9]:
time_steps = 10*20
date0 = 42680
dateT = 49985
t = np.linspace(date0, dateT, time_steps + 1)
#t = np.arange(date0, dateT, 0.25)
#time_steps = len(t) - 1

time_increment = np.ediff1d(t)/365.25
time_increment_sqrt = np.sqrt(time_increment)

# kappa
kappa = 15.0/100.0

# Local Vol 
a = 1.630971318402
b = 0.021415362451
c = 0.242030093001

sim_num = 1000
simulations = np.zeros((sim_num, 2))
extrange_brownians = np.empty((0,))
extrange_path = np.empty(0)
for path in range(sim_num):
    x = 0
    y = 0
    brownians = np.random.normal(0, scale = time_increment_sqrt)
    current_path = np.empty((0,2))
    for delta, w in zip(time_increment, brownians):
        local_vol = a*(b + c*x)
        x += (y - kappa*x)*delta + local_vol*w
        y += (local_vol**2 - 2*kappa*y)*delta
        current_path = np.vstack((current_path, [x,y]))
        #print x, y, w
        
    if not np.isfinite(x):
        extrange_brownians = brownians
        extrange_path = current_path
        
    simulations[path, :] = [x, y]

c:\python27\lib\site-packages\ipykernel\__main__.py:31: RuntimeWarning: overflow encountered in double_scalars
c:\python27\lib\site-packages\ipykernel\__main__.py:31: RuntimeWarning: invalid value encountered in double_scalars


## Modified Simulation

In [34]:
time_steps = 10*20
date0 = 42680
dateT = 49985
t = np.linspace(date0, dateT, time_steps + 1)
#t = np.arange(date0, dateT, 0.25)
#time_steps = len(t) - 1

time_increment = np.ediff1d(t)/365.25
time_increment_sqrt = np.sqrt(time_increment)

# kappa
kappa = 10.0/100.0

# Local Vol 
a = 1.008013828832
b = 0.021415362451
c = 0.303827531444

sim_num = 1000
simulations = np.zeros((sim_num, 2))
extrange_brownians = np.empty((0,))
extrange_path = np.empty(0)
for path in range(sim_num):
    x = 0
    y = 0
    brownians = np.random.normal(0, scale = time_increment_sqrt)
    current_path = np.empty((0,2))
    for delta, w in zip(time_increment, brownians):
        local_vol = a*(b + c*x)
        x += np.exp(-delta*kappa)*(y*delta + local_vol*w)
        y += (local_vol**2 - 2*kappa*y)*delta
        current_path = np.vstack((current_path, [x,y]))
        #print x, y, w
        
    if not np.isfinite(x):
        extrange_brownians = brownians
        extrange_path = current_path
        
    simulations[path, :] = [x, y]

c:\python27\lib\site-packages\ipykernel\__main__.py:31: RuntimeWarning: overflow encountered in double_scalars
c:\python27\lib\site-packages\ipykernel\__main__.py:31: RuntimeWarning: invalid value encountered in double_scalars


## Clean simulations of NaN

In [10]:
simulations.max(axis = 0)

array([ nan,  nan])

In [11]:
extrange_brownians

array([ 0.28569605,  0.33588187, -0.00138076, -0.11944366,  0.68254378,
        0.09164283, -0.24371046,  0.57077138,  0.04935418, -0.01599648,
       -0.1458389 , -0.38972155,  0.21391043,  0.18114345, -0.51843249,
        0.20024247,  0.1839545 , -0.03324305,  0.5976847 ,  0.0350407 ,
        0.52997282, -0.02355868,  0.0996549 ,  0.76559248, -0.06631759,
        0.48910508,  0.04532848, -0.18322463, -0.41794213, -0.07263862,
        0.11085037,  0.24763436,  0.22461829,  0.21299734, -0.46768064,
        0.33594472,  0.24596446,  0.2790287 ,  0.73206883,  0.01479979,
        0.20294951,  0.70980537,  0.22876859,  0.14397543,  0.16847713,
       -0.24858556, -0.19847248,  0.2347012 ,  0.30573767,  0.00482132,
       -0.55348499, -0.08213763,  0.37419508, -0.40710958, -0.03696506,
       -0.08132787, -0.20373228,  0.141032  , -0.03964314,  0.37433766,
       -0.05835395, -0.21521069,  0.5049292 ,  0.14093798, -0.07632755,
       -0.09988902, -0.13975958,  0.12818539, -0.03580919, -0.21

In [31]:
(np.isnan(simulations).sum(axis = 1) != 0).sum()

3

In [ ]:
isNan = np.isnan(simulations).sum(axis = 1) == 1
isInf = np.isinf(simulations).sum(axis = 1) == 1
simulations = simulations[np.isfinite(simulations).prod(axis = 1) == 1]
simulations = simulations[(simulations < 100).prod(axis = 1)]

## Plot results

In [ ]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

plt.close('all')
f, axarr = plt.subplots(2)

num_bins = 50
n, bins, patches = axarr[0].hist(simulations[:, 0], num_bins, normed=1, facecolor='green', alpha=0.5)
axarr[0].set_title('X Histogram')
n, bins, patches = axarr[1].hist(simulations[:, 1], num_bins, normed=1, facecolor='green', alpha=0.5)
axarr[1].set_title('Y Histogram')
plt.show()

## Calculate moments

In [ ]:
greater_moments = np.array([scipy.stats.moment(simulations, moment=x, axis=0) for x in (2,3)])
means = np.mean(simulations, 0)
moments = np.vstack((means, greater_moments))
moments[1,1] = np.cov(simulations[:,0], simulations[:,1])[0,1]
print "first moment: ", means
print "second moment: ", greater_moments[0,:]
print "third moment: ", greater_moments[1,:]

In [ ]:
experimental_values = getExcelFrame().reshape(2,3).T

In [ ]:
experimental_values

In [ ]:
significan_level = 1.0000000827403710e-011
norm_value = scipy.stats.norm.ppf([significan_level/2.0, 1.0 - significan_level/2.0])
simulations_quantiles = np.percentile(simulations, [100*significan_level/2.0, 100*(1.0 - significan_level/2.0)])

# Terminal Measure

## Simulation

In [19]:
%%time
time_steps = 24*20
date0 = 42680
dateT = 49985
dates = np.linspace(date0, dateT, time_steps + 1)
time_basis = 325.25
time_increment = np.ediff1d(dates)/time_basis
np.delete(dates, 0)

# kappa
kappa = 15.0/100.0

# Local Vol 
a = 1.630971318402
b = 0.021415362451
c = 0.242030093001

iter_no = 0
sim_num = 10000
simulations = np.zeros((sim_num, 2))
extrange_brownians = np.empty((0,))
extrange_path = np.empty(0)
for path in range(sim_num):
    for sub_path in range(hypercubes):
        x = 0
        y = 0
        
        brownians = np.random.normal(0, scale = np.sqrt(time_increment))
        current_path = np.empty((0,2))
        for date, delta, w in zip(dates, time_increment, brownians):
            G = (1.0 - np.exp(-kappa*(dateT - date)/time_basis))/kappa
            local_vol = a*(b + c*x)
            x += (y - kappa*x - local_vol**2*G)*delta + local_vol*w
            y += (local_vol**2 - 2*kappa*y)*delta
            current_path = np.vstack((current_path, [x,y]))
            #print x, y, w

        if not np.isfinite(x):
            extrange_brownians = brownians
            extrange_path = current_path

        simulations[iter_no, :] = [x, y]
        iter_no = iter_no + 1

Wall time: 1h 33min 27s


In [18]:
%%time
time_steps = 24*20
date0 = 42680
dateT = 49985
dates = np.linspace(date0, dateT, time_steps + 1)
time_basis = 325.25
time_increment = np.ediff1d(dates)/time_basis
np.delete(dates, 0)

# kappa
kappa = 15.0/100.0

# Local Vol 
a = 1.630971318402
b = 0.021415362451
c = 0.242030093001

sim_num = 100000
hypercubes = 20
sub
simulations = np.zeros((sim_num, 2))
extrange_brownians = np.empty((0,))
extrange_path = np.empty(0)
for path in range(sim_num):
    x = 0
    y = 0
    brownians = np.random.normal(0, scale = np.sqrt(time_increment))
    current_path = np.empty((0,2))
    for date, delta, w in zip(dates, time_increment, brownians):
        G = (1.0 - np.exp(-kappa*(dateT - date)/time_basis))/kappa
        local_vol = a*(b + c*x)
        x += (y - kappa*x - local_vol**2*G)*delta + local_vol*w
        y += (local_vol**2 - 2*kappa*y)*delta
        current_path = np.vstack((current_path, [x,y]))
        #print x, y, w
        
    if not np.isfinite(x):
        extrange_brownians = brownians
        extrange_path = current_path
        
    simulations[path, :] = [x, y]

NameError: name 'sub' is not defined

In [14]:
simulations.max(axis = 0)

array([ 1.34523382,  0.22504383])

In [15]:
extrange_path

array([], dtype=float64)

In [20]:
%%time
pd.DataFrame(simulations).to_csv(r"C:\Users\e022434\cheyette2.csv" , index = False, header = False, mode = 'w+')

Wall time: 1.97 s


# Simulation under Terminal Measure

## Simulation

### Results
The table of contents is automatically updated when modifications occur in the notebook. The toc window can be moved and resized. It can be docked as a sidebar or dragged from the sidebar into a floating window. The table of contents can be collapsed or the window can be completely hidden. The navigation menu can be enabled/disabled via the nbextensions configuration utility. It can also be resized. The position, dimensions, and states (that is 'collapsed' and 'hidden' states) are remembered (actually stored in the notebook's metadata) and restored on the next session. The toc window also provides two links in its header for further functionalities:

The table of contents is automatically updated when modifications occur in the notebook. The toc window can be moved and resized. It can be docked as a sidebar or dragged from the sidebar into a floating window. The table of contents can be collapsed or the window can be completely hidden. The navigation menu can be enabled/disabled via the nbextensions configuration utility. It can also be resized. The position, dimensions, and states (that is 'collapsed' and 'hidden' states) are remembered (actually stored in the notebook's metadata) and restored on the next session. The toc window also provides two links in its header for further functionalities:

In [2]:
time_steps = 100*20
date0 = 42680
dateT = 49985
dates = np.linspace(date0, dateT, time_steps + 1)
time_basis = 325.25
time_increment = np.ediff1d(dates)/time_basis
np.delete(dates, 0)

# kappa
kappa = 3.0/100.0

# Local Vol 
a = 0.390061086639
b = 0.021415362451
c = 0.456175152234

sim_num = 10000
simulations = np.zeros((sim_num, 2))
extrange_brownians = np.empty((0,))
extrange_path = np.empty(0)
for path in range(sim_num):
    x = 0
    y = 0
    brownians = np.random.normal(0, scale = np.sqrt(time_increment))
    current_path = np.empty((0,2))
    for date, delta, w in zip(dates, time_increment, brownians):
        G = (1.0 - np.exp(-kappa*(dateT - date)/time_basis))/kappa
        local_vol = a*(b + c*x)
        x += (y - kappa*x - local_vol**2*G)*delta + local_vol*w
        y += (local_vol**2 - 2*kappa*y)*delta
        current_path = np.vstack((current_path, [x,y]))
        #print x, y, w
        
    if not np.isfinite(x):
        extrange_brownians = brownians
        extrange_path = current_path
        
    simulations[path, :] = [x, y]

In [1]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %install_default_config  %install_ext  %install_profiles  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%latex  %%perl  %

In [85]:
NPV = getExcelFrame()

In [86]:
NPV.std()

20775330.136527173

In [42]:
NPV.std()

29924728.42539531

In [88]:
print NPV.mean()

11629596.8364


In [57]:
npvs = getExcelFrame()

In [62]:
npvs.std()/100000000

0.29430787996632302

In [63]:
npvs.mean()/100000000

-0.00062986217636317929

In [64]:
simulations.shape

(1000000L, 2L)

In [74]:
G_cte = 
valor = np.exp(-G_cte*simulations[:,0] - 0.5*G_cte**2*simulations[:,1])

In [69]:
valor.shape

(1000000L,)

In [76]:
valor.std()

0.32456040760821725

In [80]:
kappa = 10.0/100 
G2 = (1.0 - np.exp(-kappa*(20)))/kappa

In [81]:
G2

8.6466471676338728